In [1]:
from pathlib import Path
from pyraphtory.context import PyRaphtory
from pyraphtory.vertex import Vertex
!curl -o /tmp/lotr.csv https://raw.githubusercontent.com/Raphtory/Data/main/lotr.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 52206  100 52206    0     0   362k      0 --:--:-- --:--:-- --:--:--  383k


In [2]:
pr = PyRaphtory(spout_input=Path('/tmp/lotr.csv'), builder_script=Path('builder.py'),  builder_class='LotrGraphBuilder', mode='batch', logging=False).open()
rg = pr.graph()

In [6]:
cols = ["inDegree", "outDegree", "degree","triangleCount","prlabel","cclabel","twoHopPaths"]


df = rg.at(32674) \
                .past() \
                .transform(pr.algorithms.generic.centrality.PageRank())\
                .transform(pr.algorithms.generic.ConnectedComponents)\
                .transform(pr.algorithms.generic.centrality.Degree())\
                .transform(pr.algorithms.generic.TwoHopPaths(set([])))\
                .transform(pr.algorithms.generic.motif.LocalTriangleCount())\
                .execute(pr.algorithms.generic.NodeList(cols)) \
                .write_to_dataframe(["name"] + cols)

df

,timestamp,window,name,inDegree,outDegree,degree,triangleCount,prlabel,cclabel,twoHopPaths
0,32674,None,Hirgon,0,2,2,1,0.150000,-8637342647242242534,"[[Gandalf, Galadriel], [Gandalf, Glóin], [Gand..."
1,32674,None,Hador,2,1,3,3,0.248073,-8637342647242242534,"[[Húrin, Elessar], [Húrin, Faramir]]"
2,32674,None,Horn,3,1,4,6,0.281897,-8637342647242242534,None
3,32674,None,Galadriel,16,6,19,93,1.202763,-8637342647242242534,"[[Sam, Mablung], [Sam, Halfast], [Sam, Peregri..."
4,32674,None,Isildur,0,18,18,55,0.150000,-8637342647242242534,"[[Frodo, Mablung], [Frodo, Galadriel], [Elrond..."
...,...,...,...,...,...,...,...,...,...,...
134,32674,None,Faramir,29,3,29,132,4.614496,-8637342647242242534,"[[Elrond, Galadriel], [Imrahil, Glóin], [Imrah..."
135,32674,None,Bain,1,1,2,1,0.213750,-6628080393138316116,None
136,32674,None,Walda,10,3,13,78,0.440985,-8637342647242242534,"[[Thengel, Gram], [Folca, Gram], [Folca, Theng..."
137,32674,None,Thranduil,2,0,2,1,0.411049,-8637342647242242534,None


In [6]:
pr.shutdown()